In [1]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Tout le travail API


In [2]:
#API key
api_key = "RGAPI-6e178b98-ed70-42db-be1b-1c60e459366f" #Compte Slylio 
api_key2 = "RGAPI-1fbe27bd-678c-416f-a37c-bffeea23508d" #Compte emilio
api_key3 = "RGAPI-26cbf7f1-67e3-43ca-ac1a-5e040aa115a0" #Compte projetML
api_key5 = "RGAPI-a5972672-c708-4f9d-9fa3-2908e0d52c0e" #Comte alex1
api_key6 = "RGAPI-9dd5b684-91d0-4a69-bf6e-c945b31129f4" #Compte alex2
api_key7 = "RGAPI-46165a21-b349-4a50-9e4b-9bf71bc5d784" #Compte louis

In [4]:
"""_summary_
    This function returns the summonerId of master + ranks players in europe west.
    
    _description_
    This function returns the summonerId of the player by using the API key.
    
    _parameters_
    api_key : str
        The API key that is used to authenticate the request
"""

def get_summonerId(api_key):
    challenger_url = 'https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Challanger ENDPOINT
    grandmaster_url = 'https://euw1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Grandmaster EDNPOINT
    master_url = 'https://euw1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Master ENDPOINT

    league_url_lists = [challenger_url, grandmaster_url, master_url] # Creating a list with the 3 endpoints (master, grandmaster and challenger)

    df_list = [] # A list that will store the response from the get request (league_df)

    for url in league_url_lists: # Loop interation through league_url_lists
        resp = requests.get(url) # Sending a get request to the url
        league_df = pd.json_normalize(resp.json()['entries'])
        df_list.append(league_df)
    
    final_league_df = pd.concat(df_list) # Concatenating the stored information in league_url_lists to create the final_league_df

    return final_league_df

In [5]:
league_df = get_summonerId(api_key)
print('league_df has information about {} players between master and challenger elo'.format(league_df.shape[0]))

league_df has information about 18534 players between master and challenger elo


In [6]:
league_df

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,u6Bdbkr1xcBTGstrHuVIZEYTZMhEcF-h0i8qGDfDSbaFrzWm,GR0OT,865,I,121,76,False,False,True,False
1,Rx2c9g_K24WyGiyQEIDNdugkrCe__n8NkTrhoDIcv_Vf7gwe,KC NEXT ADKING,1277,I,230,169,True,False,False,False
2,qC4feq29rhMXT4EMEuYwRlwcjIVccejAB1Iludt8kD7QKXlM,Drali ex machina,870,I,148,102,False,False,False,True
3,w5ntb8Flx6kzI_haDWqDpnFZzpKgHm79xQzC9tg-IRc86a...,VΙadi,1298,I,344,274,True,False,False,True
4,SkdZLf_kOsNwBnIGyuUhLmb5hm9eTbjsHd-oz-mbyp2WKIag,BERFKNGKAN,958,I,136,90,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
17529,jpBD2jOcS2NiYWG2JUcJSpIa_S66Dl6kY6pPmkXhFybbez...,I SyntoX I,2,I,43,42,False,False,True,False
17530,o5YnRVvQrVPK4DWQl_04cL_IskGL5zVwc7Mv6bhVpNQH93c,Emdiel,247,I,109,98,False,False,False,True
17531,qQbVe1zUsy6OuTJN_jEGhPsHTjdBDfjSPyD1AGAtPCuITBQ,Wınged Dragon Ra,0,I,109,124,True,False,False,False
17532,XZ7dAOAePssnNhC3s_KApEGbquo8p3wyZ4UkQwF7GmQ4YoI,M6msu,235,I,104,76,False,False,False,False


In [7]:
def get_summonerinfo_by_summonerId(df, api_key):
    
    summoner_list = [] # list that will store the each summoner_info DataFrame

    for i in range(len(df)): # len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        # Creating a dynamic api_ulr for each register in the summonerId column using iloc
        api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
        print(api_url)
        # Repeating the process we did at the begining
        resp = requests.get(api_url)
        print(resp.status_code)

        # if, elif and while true to deal with riot api rate limit
        if resp.status_code == 200:
            pass
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 10 second') #approximate 10 second wait before trying again
                    time.sleep(10)

                    # Trying again
                    print(api_url)
                    api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break

        summoner_info = pd.json_normalize(resp.json()) # The same process we did at the beggining to create the DataFrame
        summoner_list.append(summoner_info) # Appending the DataFrame generated (summoner_info) into the list "summoner_list"
        
    final_summoner_info = pd.concat(summoner_list) # Using pd.concat to concatenate the results stored in the list "summoner_list"
            
    return final_summoner_info
    

In [7]:
"""
summoner_info = get_summonerinfo_by_summonerId(league_df[:1500], api_key)
summoner_info2 = get_summonerinfo_by_summonerId(league_df[1500:3000], api_key)
summoner_info3 = get_summonerinfo_by_summonerId(league_df[3000:5500], api_key)
summoner_info4 = get_summonerinfo_by_summonerId(league_df[5500:8000], api_key)
summoner_info5 = get_summonerinfo_by_summonerId(league_df[8000:10500], api_key)
summoner_info = pd.concat([summoner_info, summoner_info2, summoner_info3, summoner_info4, summoner_info5])
summoner_info.drop_duplicates(subset=['accountId'], inplace=True)
summoner_info.to_csv('summoner_info.csv', index=False)
"""

"\nsummoner_info = get_summonerinfo_by_summonerId(league_df[:1500], api_key)\nsummoner_info2 = get_summonerinfo_by_summonerId(league_df[1500:3000], api_key)\nsummoner_info3 = get_summonerinfo_by_summonerId(league_df[3000:5500], api_key)\nsummoner_info4 = get_summonerinfo_by_summonerId(league_df[5500:8000], api_key)\nsummoner_info5 = get_summonerinfo_by_summonerId(league_df[8000:10500], api_key)\nsummoner_info = pd.concat([summoner_info, summoner_info2, summoner_info3, summoner_info4, summoner_info5])\nsummoner_info.drop_duplicates(subset=['accountId'], inplace=True)\nsummoner_info.to_csv('summoner_info.csv', index=False)\n"

In [8]:
def get_matchids_by_puuid(df, api_key):

    matchids_list = []

    for i in range(len(df)):

        api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=50&api_key={}'.format(df['puuid'].iloc[i], api_key)

        resp = requests.get(api_url) # Sending a get request to the api_url
        print(resp.json())

        if resp.status_code == 200:
            matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
            matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 100 second') #approximate 110 second wait
                    time.sleep(10)

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=50&api_key={}'.format(df['puuid'].iloc[i], api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                    if resp.status_code == 200:
                        matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
                        matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
                        print('limit cost resolve')
                        break
                else:
                    break

    matchids_df = pd.concat(matchids_list) # Using pd.concat to concatenate the results stored in the "matchids_list"
    
    return matchids_df


In [ ]:
#Remove duplicates
size_before = matchids_df.shape[0]
matchids_df = matchids_df.drop_duplicates()
size_after = matchids_df.shape[0]
print('Removed {} duplicates'.format(size_before - size_after))
print('matchids_df has information about {} matches'.format(matchids_df.shape[0]))

NameError: name 'matchids_df' is not defined

In [ ]:
matchids_df.to_csv('matchids.csv', index=False)

NameError: name 'matchids_df' is not defined

In [ ]:
matchids_df.reset_index(inplace=True)
matchids_df.drop(columns = 'index', inplace = True )
matchids_df

NameError: name 'matchids_df' is not defined

In [9]:
def get_match_info_by_matchId(match_ids, api_key):

    matchid_info_list = [] # List that will store the dataframes generated, for each match id in the for loop 

    for i in range(len(match_ids)):# len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        api_ulr = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}/timeline?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # here we use the iloc function as explained above to form the api link for each matchid
        resp = requests.get(api_ulr) # Sending a get request to the api_url
        print(resp.status_code) # Printing the status_code of the request

        if resp.status_code == 200: # Here we need to use if statment, because of the riot API rate limit which is 20 requests every 1 sec and 100 requests every 2 minutes. A 200 status_code means that the request was successful so it can move on.
            pass                        
        elif resp.status_code == 429: # A 429 means that we have exceeded the rate limit. To solve that we need to wait and then try again to get a successful request
            while True: # To do taht we use a while true loop, to execute the if, elif statment repeatdly until the status_code is 200. After that we use break statement to come out of this loop.
                if resp.status_code == 429: 
                    print('429 delay try 10 second') #approximate 110 second wait to work
                    time.sleep(10) # Sleep

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # Trying the request again
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break        

        # Same process we did in the beginning             
        matchid_info = pd.json_normalize(resp.json())
        matchid_info_list.append(matchid_info) # Merging all the dataframes generated and then stored in the matchid_info_list with the concat() function
        
    matchid_info_df = pd.concat(matchid_info_list)

    return matchid_info_df

In [10]:
from concurrent.futures import ThreadPoolExecutor

In [11]:
matchs_ids = pd.read_csv('matchids.csv')

In [12]:
matchs_ids

,matchid
0,EUW1_6872766682
1,EUW1_6872721234
2,EUW1_6872682077
3,EUW1_6872636755
4,EUW1_6872581293
...,...
237638,EUW1_6811998207
237639,EUW1_6811267636
237640,EUW1_6811199346
237641,EUW1_6810916271


In [15]:
def multi_get_match_info(matchs_ids, api_keys):
    with ThreadPoolExecutor() as executor:
        futures = []
        matchs_info_df = []
        for i in range(len(api_keys)):
            future = executor.submit(get_match_info_by_matchId, np.array_split(matchs_ids, len(api_keys))[i], api_keys[i])
            futures.append(future)
        for future in futures:
            matchs_info_df.append(future.result())
    return pd.concat(matchs_info_df)

    
#Create a 5Gb file
df_matchs_info1 = multi_get_match_info(matchs_ids[:5000], [api_key, api_key2, api_key3, api_key5, api_key6, api_key7])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
429
429 delay try 10 second
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
429
429 delay try 10 second


In [16]:
df_matchs_info1.shape
df_matchs_info1

,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.frameInterval,info.frames,info.gameId,info.participants,info.gameCreation,info.gameDuration,...,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.platformId,info.queueId,info.teams,info.tournamentCode
0,2,EUW1_6872766682,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711467506447, ...",6872766682,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,EUW1_6872721234,[nFVx2BkBRefMe_OoGq9ATC9zKDtMRxC9s8PGd8oTu61aL...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711465572396, ...",6872721234,"[{'participantId': 1, 'puuid': 'nFVx2BkBRefMe_...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,EUW1_6872682077,[lreOyIqCn3xo20ipRzSwUTvwqW8eFxA0l8iXGni3OM9Rp...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711463416185, ...",6872682077,"[{'participantId': 1, 'puuid': 'lreOyIqCn3xo20...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,EUW1_6872636755,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711460683572, ...",6872636755,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,EUW1_6872581293,[wCdvPlukkQSkihDiNIfb5Bkez12fl-hScsUtlhCWgJ1Ib...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711457894967, ...",6872581293,"[{'participantId': 1, 'puuid': 'wCdvPlukkQSkih...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2,EUW1_6872492425,[V6ovA4OBpOkX2QY5snGATD2npNl9J3kSD3PM4cd33SzX9...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711451950819, ...",6872492425,"[{'participantId': 1, 'puuid': 'V6ovA4OBpOkX2Q...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,EUW1_6867829052,[5D00eB4pOFbcVFl9BVGUrozor31kWKYqetCTUwgqlzDEl...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711190683074, ...",6867829052,"[{'participantId': 1, 'puuid': '5D00eB4pOFbcVF...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,EUW1_6867799491,[YxS8hXA90JOvfQ3bJwEfgmdsb-KRjf5mDvayklwBapifg...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711188500434, ...",6867799491,"[{'participantId': 1, 'puuid': 'YxS8hXA90JOvfQ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,EUW1_6862658508,[NM4KknHSDXpISu9PPlP_xW2ZUsVr3-bNeZWwD4k9AKAYV...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1710873655741, ...",6862658508,"[{'participantId': 1, 'puuid': 'NM4KknHSDXpISu...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
matchs_info_df_index = df_matchs_info1.reset_index()
matchs_info_df_index

,index,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.frameInterval,info.frames,info.gameId,info.participants,info.gameCreation,...,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.platformId,info.queueId,info.teams,info.tournamentCode
0,0,2,EUW1_6872766682,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711467506447, ...",6872766682,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2,EUW1_6872721234,[nFVx2BkBRefMe_OoGq9ATC9zKDtMRxC9s8PGd8oTu61aL...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711465572396, ...",6872721234,"[{'participantId': 1, 'puuid': 'nFVx2BkBRefMe_...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2,EUW1_6872682077,[lreOyIqCn3xo20ipRzSwUTvwqW8eFxA0l8iXGni3OM9Rp...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711463416185, ...",6872682077,"[{'participantId': 1, 'puuid': 'lreOyIqCn3xo20...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2,EUW1_6872636755,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711460683572, ...",6872636755,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2,EUW1_6872581293,[wCdvPlukkQSkihDiNIfb5Bkez12fl-hScsUtlhCWgJ1Ib...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711457894967, ...",6872581293,"[{'participantId': 1, 'puuid': 'wCdvPlukkQSkih...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,2,EUW1_6872492425,[V6ovA4OBpOkX2QY5snGATD2npNl9J3kSD3PM4cd33SzX9...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711451950819, ...",6872492425,"[{'participantId': 1, 'puuid': 'V6ovA4OBpOkX2Q...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,0,2,EUW1_6867829052,[5D00eB4pOFbcVFl9BVGUrozor31kWKYqetCTUwgqlzDEl...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711190683074, ...",6867829052,"[{'participantId': 1, 'puuid': '5D00eB4pOFbcVF...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,0,2,EUW1_6867799491,[YxS8hXA90JOvfQ3bJwEfgmdsb-KRjf5mDvayklwBapifg...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1711188500434, ...",6867799491,"[{'participantId': 1, 'puuid': 'YxS8hXA90JOvfQ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,0,2,EUW1_6862658508,[NM4KknHSDXpISu9PPlP_xW2ZUsVr3-bNeZWwD4k9AKAYV...,GameComplete,60000.0,"[{'events': [{'realTimestamp': 1710873655741, ...",6862658508,"[{'participantId': 1, 'puuid': 'NM4KknHSDXpISu...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
dataset = pd.DataFrame()
for i in range(len(matchs_info_df_index)):
    print(i)
    end = pd.DataFrame(pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"])-1]).iloc[len(pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"][len(pd.DataFrame(matchs_info_df_index["info.frames"][i])["events"])-1]))-1]).transpose()
    end["winningTeam"] = int((end["winningTeam"]-100)/100)
    end = pd.DataFrame(end.reset_index()["winningTeam"])

    match_data_stats = pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][i])["participantFrames"])
    for j in range(11):
        match_data_stats.columns = match_data_stats.columns.str.removeprefix(str(j) + ".")
    team1_stats = match_data_stats.iloc[:, 1:240]
    team2_stats = match_data_stats.iloc[:, 240:]
    team1_stats = team1_stats.groupby(team1_stats.columns, axis=1).sum()
    team1_stats = team1_stats.add_prefix("Team1.")
    team2_stats = team2_stats.groupby(team2_stats.columns, axis=1).sum()
    team2_stats = team2_stats.add_prefix("Team2.")
    match_data_stats_teams = pd.concat([team1_stats, team2_stats], axis=1)
    match_data_stats_teams
    match_data_stats_teams_15 = match_data_stats_teams.loc[15]
    match_data_stats_teams_15= pd.DataFrame(match_data_stats_teams_15).transpose().reset_index().drop(["index"], axis=1)
    pd.concat([end, match_data_stats_teams_15], axis=1)
    dataset = pd.concat([dataset, pd.concat([end, match_data_stats_teams_15], axis=1)], axis = 0)
dataset

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


ValueError: DataFrame constructor not properly called!

In [128]:
dataset.to_csv("maches_15m_data.csv", index = False)